# **Análise de Crédito e Inadimplência**

Projeto final do módulo "Python para Análise de Dados", parte integrante do curso de Ciência de Dados da EBAC.


# **Etapas**

<ol type="1">
  <li>Contexto</li>
  <li>Análise Exploratória</li>
  <li>Preparação dos Dados</li>
  <li>Visualização e Insights</li>
  <li>Conclusões</li>
</ol>


# 1\. Contexto

O objetivo desta análise é investigar os fatores que contribuem para a inadimplência de um cliente, considerando o comportamento de alguns de seus atributos, como renda, relacionamento com a empresa e histórico financeiro.

Para isso, vamos explorar os dados de crédito disponíveis no seguinte [link](https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/develop/dataset/credito.csv). Esses dados estão em formato CSV e fornecem informações sobre os clientes de uma instituição financeira. (Fornecidos por [André Perez](https://www.linkedin.com/in/andremarcosperez/))

Especificamente, concentraremos nossa atenção na segunda coluna, denominada "default", que indica se um cliente é adimplente (default = 0) ou inadimplente (default = 1).

Abaixo está uma descrição completa dos atributos presentes no  arquivo CSV:

| Coluna  | Descrição |
| ------- | --------- |
| id      | Número da conta |
| default | Indica se o cliente é adimplente (0) ou inadimplente (1) |
| idade   | Idade do cliente |
| sexo    | Gênero do cliente |
| dependentes | Número de dependentes financeiros |
| escolaridade | Nível de escolaridade |
| estado_civil | Estado civil |
| salario_anual | Faixa do salário mensal multiplicado por 12 |
| tipo_cartao | Categoria do cartão |
| meses_de_relacionamento | Tempo de relacionamento com a instituição |
| qtd_produtos | Quantidade de produtos contratados |
| iteracoes_12m | Quantidade de iterações com o cliente no último ano |
| meses_inativo_12m | Quantidade de meses que o cliente ficou inativo no último ano |
| limite_credito | Valor do limite de crédito do cartão |
| valor_transacoes_12m | Valor total das transações no cartão de crédito no último ano |
| qtd_transacoes_12m | Quantidade total de transações no cartão de crédito no último ano |


# 2\. Análise Exploratória

Vamos iniciar importando as bibliotecas necessárias e realizando a leitura dos dados.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

sns.set(style='whitegrid')

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/develop/dataset/credito.csv', na_values='na')
df.head()

## 2\.1 Schema

Vamos examinar a estrutura e o formato dos dados

In [ ]:
linhas, colunas = df.shape
print(f'O arquivo possui {linhas} linhas e {colunas} colunas.')

In [ ]:
total_clientes = len(df)
adimplentes = df['default'].value_counts()[0]
inadimplentes = df['default'].value_counts()[1]

prop_adimplentes = round(100 * adimplentes / total_clientes, 2)
prop_inadimplentes = round(100 * inadimplentes / total_clientes, 2)

print(f'A proporção de clientes adimplentes é de {prop_adimplentes}% '
    f'({adimplentes} de {total_clientes})')
print(f'A proporção de clientes inadimplentes é de {prop_inadimplentes}% '
    f'({inadimplentes} de {total_clientes})')

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
# estatísticas descritivas das variáveis numéricas
df.describe().drop('id', axis=1)

In [ ]:
# estatísticas descritivas das variáveis categóricas
df.describe(include=['object'])

In [ ]:
# verifica se há alguma conta duplicada
df['id'].duplicated().any()

## 2\.2 Dados faltantes

A seguir, apresentaremos algumas estatísticas sobre os dados ausentes em colunas específicas.

In [ ]:
df.isna().any()

In [ ]:
def total_faltantes(df: pd.DataFrame) -> None:
    qtd_total = df.isna().sum().sum()
    prop_total = df.isna().mean().mean() * 100
    print(f'A base de dados apresenta um total de {qtd_total} dados faltantes '
    f'({prop_total:.2f}%)')

In [ ]:
# quantidade total de dados faltantes
total_faltantes(df)

In [ ]:
def dados_faltantes(df):
    dados_ausentes = df.isna().sum()
    prop_ausentes = 100 * df.isna().mean()

    for col, qtd, prop in zip(df.columns, dados_ausentes, prop_ausentes):
        if qtd > 0:
            print(f'{col}: faltam {qtd} dados ({prop:.2f}%)')

In [ ]:
# apresenta os dados faltantes por coluna
dados_faltantes(df)

In [ ]:
print('Proporção de dados faltantes dos clientes adimplentes:')
dados_faltantes(df[df['default'] == 0])

In [ ]:
print('Proporção de dados faltantes dos clientes inadimplentes:')
dados_faltantes(df[df['default'] == 1])

# 3\. Preparação dos Dados

As colunas “valor_transacoes_12m” e “limite_credito” estão sendo erroneamente interpretadas como categóricas. Vamos ajustá-las para que sejam tratadas corretamente como variáveis numéricas. Além disso, após análise, constatamos que a proporção de linhas com dados faltantes não impacta significativamente nossa base de dados. Portanto, podemos excluí-las sem grande prejuízo.

In [ ]:
def conv_moeda(df, column):
    df[column] = df[column].str.replace('.', '').str.replace(',', '.').astype(float)

In [ ]:
conv_moeda(df, 'limite_credito')
conv_moeda(df, 'valor_transacoes_12m')
df.dropna(inplace=True)

In [ ]:
# verifica se ainda há dados faltantes
df.isna().sum()

In [ ]:
df[['limite_credito', 'valor_transacoes_12m']].head()

In [ ]:
print(f"O total de linhas agora é: {len(df)}")

In [ ]:
qtd_total_novo = len(df)
qtd_adimplentes_novo, qtd_inadimplentes_novo = df.groupby('default').size()

In [ ]:
prop_adimplentes_novo = round(100 * qtd_adimplentes_novo / qtd_total_novo, 2)
prop_inadimplentes_novo = round(100 * qtd_inadimplentes_novo / qtd_total_novo, 2)

print(f'A nova proporção de clientes adimplentes é de {prop_adimplentes_novo}% '
    f'({qtd_adimplentes_novo} de {qtd_total_novo})')
print(f'A nova proporção de clientes inadimplentes é de {prop_inadimplentes_novo}% '
    f'({qtd_inadimplentes_novo} de {qtd_total_novo})')

In [ ]:
df.rename(columns={'idade' : 'Idade', 'sexo' : 'Sexo', 'dependentes' : 'Dependentes', 'escolaridade' : 'Escolaridade', 'estado_civil' : 'Estado_Civil', 'salario_anual' : 'Salário_Anual', 'tipo_cartao' : 'Tipo_Cartão', 'meses_de_relacionamento' : 'Meses_de_Relacionamento', 'qtd_produtos' : 'Quantidade_Produtos', 'iteracoes_12m' : 'Iterações_12m', 'meses_inativo_12m' : 'Meses_Inativo_12m', 'limite_credito' : 'Limite_de_Crédito', 'valor_transacoes_12m' : 'Valor_Transações_12m', 'qtd_transacoes_12m' : 'Quantidade_Transações_12m'}, inplace=True)

In [ ]:
# informações gerais sobre as mudanças realizadas
print(df.info())

# 4\. Visualização e Insights

Nesta etapa, vamos criar algumas visualizações para correlacionar os atributos que procuram explicar a coluna 'default'. Alguns dados não serão utilizados nas análises, em conformidade com a LGPD.

In [ ]:
df_adimplente = df[df['default'] == 0]

In [ ]:
df_inadimplente = df[df['default'] == 1]

In [ ]:
plt.figure(figsize=(8, 6))
plt.pie([prop_adimplentes_novo, prop_inadimplentes_novo],
        labels=['Adimplentes', 'Inadimplentes'],
        colors=['#1f77b4', '#ff7f0e'],
        autopct='%1.2f%%',
        startangle=140)
plt.title('Proporção de clientes')
plt.axis('equal')

plt.show()

## 4\.1 Visualizações Categóricas

- Escolaridade

In [ ]:
coluna = 'Escolaridade'
titulos = ['Total de Clientes', 'Adimplentes', 'Inadimplentes']

figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharey=True)

for eixo, dataframe in enumerate([df, df_adimplente, df_inadimplente]):
    frequencia_absoluta = dataframe[coluna].value_counts()
    sns.barplot(x=frequencia_absoluta.index, y=frequencia_absoluta.values, ax=eixos[eixo])
    eixos[eixo].set(title=titulos[eixo], xlabel=coluna, ylabel='Frequência Absoluta')
    eixos[eixo].tick_params(axis='x', rotation=90)

plt.show()

- Estado Civil

In [ ]:
coluna = 'Estado_Civil'
titulos = ['Total de Clientes', 'Adimplentes', 'Inadimplentes']

figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharey=True)

for eixo, dataframe in enumerate([df, df_adimplente, df_inadimplente]):
    frequencia_absoluta = dataframe[coluna].value_counts()
    sns.barplot(x=frequencia_absoluta.index, y=frequencia_absoluta.values, ax=eixos[eixo])
    eixos[eixo].set(title=titulos[eixo], xlabel=coluna, ylabel='Frequência Absoluta')
    eixos[eixo].tick_params(axis='x', rotation=90)

plt.show()

- Salário Anual

In [ ]:
coluna = 'Salário_Anual'
titulos = ['Total de Clientes', 'Adimplentes', 'Inadimplentes']

figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharey=True)

for eixo, dataframe in enumerate([df, df_adimplente, df_inadimplente]):
    frequencia_absoluta = dataframe[coluna].value_counts()
    sns.barplot(x=frequencia_absoluta.index, y=frequencia_absoluta.values, ax=eixos[eixo])
    eixos[eixo].set(title=titulos[eixo], xlabel=coluna, ylabel='Frequência Absoluta')
    eixos[eixo].tick_params(axis='x', rotation=90)

plt.show()

- Tipo do Cartão

In [ ]:
coluna = 'Tipo_Cartão'
titulos = ['Total de Clientes', 'Adimplentes', 'Inadimplentes']

figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharey=True)

for eixo, dataframe in enumerate([df, df_adimplente, df_inadimplente]):
    frequencia_absoluta = dataframe[coluna].value_counts()
    sns.barplot(x=frequencia_absoluta.index, y=frequencia_absoluta.values, ax=eixos[eixo])
    eixos[eixo].set(title=titulos[eixo], xlabel=coluna, ylabel='Frequência Absoluta')
    eixos[eixo].tick_params(axis='x', rotation=90)

plt.show()

## 4\.2 Visualizações Numéricas

In [ ]:
df_corr = df[['Meses_de_Relacionamento', 'Idade', 'Dependentes', 
                'Quantidade_Produtos', 'Iterações_12m','Meses_Inativo_12m', 
                'Limite_de_Crédito', 'Valor_Transações_12m',
                'Quantidade_Transações_12m']].corr()
df_corr

In [ ]:
sns.heatmap(df_corr, annot=True, cmap='coolwarm')

- Idade

In [ ]:
coluna = 'Idade'
titulos = ['Total de Clientes', 'Adimplentes', 'Inadimplentes']

figura, eixos = plt.subplots(1, 3, figsize=(20, 5), sharey=True)

for eixo, dataframe in enumerate([df, df_adimplente, df_inadimplente]):
    f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=eixos[eixo])
    f.set(title=titulos[eixo], xlabel=coluna, ylabel='Frequência Absoluta')
    f.set_ylim(0, None)

plt.show()

- Meses de Relacionamento

In [ ]:
coluna = 'Meses_de_Relacionamento'
titulos = ['Total de Clientes', 'Adimplentes', 'Inadimplentes']

figura, eixos = plt.subplots(1, 3, figsize=(20, 5), sharey=True)

for eixo, dataframe in enumerate([df, df_adimplente, df_inadimplente]):
    f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=eixos[eixo])
    f.set(title=titulos[eixo], xlabel=coluna, ylabel='Frequência Absoluta')
    f.set_ylim(0, None)

plt.show()

In [ ]:
df_sorted = df.sort_values('Idade')

plt.figure(figsize=(10, 6))
sns.lineplot(x='Idade', y='Meses_de_Relacionamento', hue='default', data=df_sorted)
plt.title('Relação entre Idade e Meses de Relacionamento')
plt.xlabel('Idade dos Clientes')
plt.ylabel('Meses de Relacionamento')
plt.legend(title='Default', loc='best')
plt.grid(True)

plt.show()

- Valor das transações nos últimos 12 meses

In [ ]:
coluna = 'Valor_Transações_12m'
titulos = ['Total de Clientes', 'Adimplentes', 'Inadimplentes']

figura, eixos = plt.subplots(1, 3, figsize=(20, 5), sharey=True)

for eixo, dataframe in enumerate([df, df_adimplente, df_inadimplente]):
    f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=eixos[eixo])
    f.set(title=titulos[eixo], xlabel=coluna, ylabel='Frequência Absoluta')
    f.set_ylim(0, None)

plt.show()

- Quantidade de transações nos últimos 12 meses

In [ ]:
coluna = 'Quantidade_Transações_12m'
titulos = ['Total de Clientes', 'Adimplentes', 'Inadimplentes']

figura, eixos = plt.subplots(1, 3, figsize=(20, 5), sharey=True)

for eixo, dataframe in enumerate([df, df_adimplente, df_inadimplente]):
    f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=eixos[eixo])
    f.set(title=titulos[eixo], xlabel=coluna, ylabel='Frequência Absoluta')
    f.set_ylim(0, None)

plt.show()

In [ ]:
sns.scatterplot(x='Valor_Transações_12m', y='Quantidade_Transações_12m', data=df, hue='default')
plt.title('Relação entre Valor e Quantidade de Transações no Último Ano')
plt.xlabel('Valor das Transações no Último Ano')
plt.ylabel('Quantidade de Transações no Último Ano')

plt.show()

# 5\. Conclusões

As conclusões apresentadas a seguir derivam de uma análise detalhada dos dados fornecidos, que foram explorados e examinados com o objetivo de entender os padrões e tendências relacionados à inadimplência dos clientes. É importante ressaltar que essas conclusões refletem apenas os resultados obtidos a partir da análise do banco de dados disponível e não devem ser interpretadas como indicadores de total causalidade. Embora forneçam insights valiosos sobre os fatores associados à inadimplência, outras variáveis e fatores externos podem influenciar os resultados observados. Portanto, essas conclusões devem ser consideradas como parte de um quadro mais amplo e sujeitas a investigações adicionais e análises aprofundadas.

- Ao analisar as variáveis categóricas (Escolaridade, Estado Civil, Salário Anual e Limite do Cartão), não foi possível identificar nenhum padrão que justificasse a inadimplência dos clientes.

-  Observou-se uma tendência de aumento do número de meses de relacionamento à medida que a idade dos clientes aumenta. Isso sugere que clientes com mais idade tendem a ter um relacionamento mais longo com a empresa.

- Verificou-se uma tendência de inadimplência entre os clientes com até 30 anos de idade e menos de 40 meses de relacionamento com a instituição financeira.

- Ao analisar as quantidades de transações nos últimos 12 meses, percebemos que os clientes que realizaram entre 20 e 60 transações tendem a se tornar inadimplentes.

- A análise dos valores de transações nos últimos 12 meses indicou que os clientes com transações entre R$ 1.000 e R$ 3.000 estão mais propensos à inadimplência.

- Notou-se que a maioria dos clientes com 60 a 80 transações no último ano e um valor de transações entre R$ 7.500 e R$ 10.000 são inadimplentes.

- Por outro lado, clientes com mais de 100 transações no último ano e com valores de transações superiores a R$ 12.000 não indicam nenhum sinal de inadimplência.
